In [ ]:
from model import MLPNet
import tf_keras as keras
import pandas as pd
import tensorflow as tf
import crystal_loader
from tqdm import tqdm
import tqdm.keras
# import symmetry
from symmetry import *
import dill
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split


In [ ]:
fpath = "../data/TiO2/"

structs, labels = crystal_loader.get_dataset(fpath, halt=-1)

with open("TiO2_features.dill", "rb") as dill_file:
    features = dill.load(dill_file)

In [ ]:
layers = [keras.layers.Dense(100, activation="relu"), keras.layers.Dense(100, activation="relu")] + list([keras.layers.Dense(50, activation="relu") for i in range(10)])

MLP1 = MLPNet(layers=layers, n_syms=13)

MLP1.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    loss = keras.losses.MeanAbsoluteError(),
    metrics = [keras.metrics.RootMeanSquaredError()]
)


In [ ]:
def scale_ragged(features):
    stacked = np.hstack(features).T
    SSC = StandardScaler().fit(stacked)
    scaled_features = [SSC.transform(struct.T) for struct in features]

    return scaled_features


print(scale_ragged(features))

In [ ]:
Xtrain = tf.ragged.constant(features, ragged_rank=1, inner_shape=(13,))

In [ ]:
res = MLP1.fit(
    Xtrain, labels,
    batch_size = 50,
    epochs = 200,
    # validation_data = (Xval, yval),
    verbose = 0,
    callbacks=[tqdm.keras.TqdmCallback()]
)

In [ ]:
print(MLP1.get_subnet().predict(Xtrain[0]))